In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transforms as transforms
torch.__version__

'1.0.1'

In [3]:
class ACNN(nn.Module):

    def __init__(self, n_classes):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        super(ACNN, self).__init__()
        
        # kernel
        # 1 input image channel, 10 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(1, 10, 5)
        # 10 input image channel, 10 output channels, 5x5 square convolution
        self.conv2 = nn.Conv2d(10, 10, 5)
        # 10 input image channel, 10 output channels, 3x3 square convolution
        self.conv3 = nn.Conv2d(10, 10, 3)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(10 * 3 * 3, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, n_classes)
        
        self.softmax = nn.Softmax(1)

    def forward(self, x):
        # x size [BATCHSIZE, 1, 48, 48]
#         print("0:", x.size())
        x = F.avg_pool2d(F.relu(self.conv1(x)), (2, 2)) # x size [BATCHSIZE, 10, 22, 22]
#         print("1:", x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # x size [BATCHSIZE, 10, 9, 9]
#         print("2:", x.size())
        x = F.max_pool2d(F.relu(self.conv3(x)), 2) # x size [BATCHSIZE, 10, 3, 3]
#         print("3:", x.size())
        #这里做的就是压扁的操作 就是把后面的[BATCHSIZE, 10, 3, 3]压扁，变为 [4, 90]
        x = x.view(-1, self.num_flat_features(x))
#         print("3:", x.size())
        # 输入为 144
        x = F.relu(self.fc1(x))  # x size [BATCHSIZE, 64]
#         print("11:", x.size())
        x = F.relu(self.fc2(x))  # x size [BATCHSIZE, 16]
#         print("12:", x.size())
        x = self.fc3(x)  # x size [BATCHSIZE, n_classes]
#         print("13:", x.size())
        x = self.softmax(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

input_img_size = 48
n_classes=7
net = ACNN(n_classes=n_classes)
print(net)

ACNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=90, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=7, bias=True)
  (softmax): Softmax()
)


In [4]:
# for parameters in net.parameters():
#     print(parameters.size())
#     print(parameters)

for name,parameters in net.named_parameters():
    print(name,':',parameters.size())
    print(parameters)

conv1.weight : torch.Size([10, 1, 5, 5])
Parameter containing:
tensor([[[[-0.1261, -0.1346, -0.0517,  0.1388,  0.1194],
          [-0.0083,  0.1856, -0.0768, -0.1629, -0.1844],
          [-0.0483,  0.1212, -0.1038,  0.1004,  0.1688],
          [ 0.1908, -0.1105, -0.0410, -0.1364,  0.1503],
          [-0.1394, -0.1518, -0.0154,  0.1782, -0.1352]]],


        [[[-0.0101,  0.0567, -0.1571,  0.0862, -0.0314],
          [-0.0806,  0.0236,  0.0558,  0.0017,  0.0920],
          [ 0.0619, -0.1345,  0.0632,  0.0751, -0.0200],
          [ 0.0190, -0.0558,  0.1945, -0.1451, -0.1912],
          [-0.0230, -0.0959, -0.1967, -0.0344, -0.1476]]],


        [[[-0.1933, -0.1746, -0.1292,  0.1750,  0.0310],
          [-0.0431,  0.0330,  0.1814, -0.1079, -0.1981],
          [ 0.1177,  0.0475,  0.1174, -0.0092,  0.0165],
          [-0.1441, -0.0897, -0.1152,  0.0473, -0.0573],
          [ 0.1332, -0.1441,  0.1638, -0.0241, -0.0807]]],


        [[[ 0.1001, -0.1511,  0.0233,  0.0524,  0.1236],
          [ 0

In [5]:
BATCH_SIZE=4
input = torch.randn(BATCH_SIZE, 1, input_img_size, input_img_size) # 这里的对应前面fforward的输入是32
out = net(input)
out.size()


torch.Size([4, 7])

In [6]:
net.zero_grad() 
out.backward(torch.ones(BATCH_SIZE,n_classes))

In [7]:
y = torch.arange(0, BATCH_SIZE*n_classes).view(BATCH_SIZE,n_classes).float() # 这里的view其实没起什么作用，为了防止将来前面变化而代码出错0
criterion = nn.MSELoss()
loss = criterion(out, y)
#loss是个scalar，我们可以直接用item获取到他的python类型的数值
print(out)
print(y)
print(loss.item()) 

tensor([[0.1901, 0.1356, 0.1384, 0.1259, 0.1460, 0.1260, 0.1381],
        [0.1901, 0.1359, 0.1393, 0.1256, 0.1465, 0.1253, 0.1374],
        [0.1902, 0.1359, 0.1396, 0.1253, 0.1462, 0.1253, 0.1375],
        [0.1905, 0.1360, 0.1384, 0.1255, 0.1465, 0.1255, 0.1376]],
       grad_fn=<SoftmaxBackward>)
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [ 7.,  8.,  9., 10., 11., 12., 13.],
        [14., 15., 16., 17., 18., 19., 20.],
        [21., 22., 23., 24., 25., 26., 27.]])
243.7125244140625


In [8]:
import torch.optim

out = net(input) # 这里调用的时候会打印出我们在forword函数中打印的x的大小
criterion = nn.MSELoss()
loss = criterion(out, y)
#新建一个优化器，SGD只需要要调整的参数和学习率
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 
loss.backward()

#更新参数
optimizer.step()

In [9]:
for parameters in net.parameters():
    print(parameters.size())
    print(parameters)

torch.Size([10, 1, 5, 5])
Parameter containing:
tensor([[[[-0.1262, -0.1346, -0.0517,  0.1388,  0.1194],
          [-0.0083,  0.1856, -0.0768, -0.1629, -0.1844],
          [-0.0483,  0.1212, -0.1038,  0.1004,  0.1688],
          [ 0.1908, -0.1105, -0.0410, -0.1364,  0.1503],
          [-0.1394, -0.1518, -0.0154,  0.1782, -0.1352]]],


        [[[-0.0101,  0.0567, -0.1571,  0.0862, -0.0314],
          [-0.0806,  0.0236,  0.0558,  0.0017,  0.0920],
          [ 0.0619, -0.1345,  0.0632,  0.0751, -0.0199],
          [ 0.0190, -0.0558,  0.1945, -0.1451, -0.1912],
          [-0.0230, -0.0959, -0.1967, -0.0344, -0.1476]]],


        [[[-0.1933, -0.1746, -0.1292,  0.1750,  0.0310],
          [-0.0431,  0.0330,  0.1814, -0.1079, -0.1981],
          [ 0.1177,  0.0475,  0.1174, -0.0092,  0.0165],
          [-0.1441, -0.0897, -0.1152,  0.0473, -0.0573],
          [ 0.1332, -0.1441,  0.1638, -0.0241, -0.0807]]],


        [[[ 0.1001, -0.1511,  0.0233,  0.0524,  0.1236],
          [ 0.1676,  0.0228,

以下部分为读取训练集和测试集进行分类
--------------

In [10]:
BATCH_SIZE=128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多
print(DEVICE)
use_cuda = torch.cuda.is_available()

learning_rate = 0.01

cuda


In [11]:
net = ACNN(n_classes=7).to(DEVICE)

In [12]:
import os
from PIL import Image
import numpy as np
import torch.utils.data as data
import utils
from torch.autograd import Variable

In [13]:
class JAFFE(data.Dataset):
    """`JAFFE Dataset.
    Args:
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``

        there are NEU:30 HAP:31 SAD:31 SUR:30 ANG:30 DIS:29 FEA:32 images in data with ten people
        we choose images of 9 people for training
        we choose images of 1 people for testing
        the split are in order according to the fold number
    """

    def __init__(self, is_train=True, transform=None):
        self.classes_map = {'NE': np.array([1., 0., 0., 0., 0., 0., 0.]), 
                            'HA': np.array([0., 1., 0., 0., 0., 0., 0.]), 
                            'SA': np.array([0., 0., 1., 0., 0., 0., 0.]), 
                            'SU': np.array([0., 0., 0., 1., 0., 0., 0.]), 
                            'AN': np.array([0., 0., 0., 0., 1., 0., 0.]), 
                            'DI': np.array([0., 0., 0., 0., 0., 1., 0.]), 
                            'FE': np.array([0., 0., 0., 0., 0., 0., 1.])}
        self.img_dir_pre_path = "data/jaffe"
        self.train_people_names = ['MK', 'UY', 'KL', 'NM', 'YM', 'TM', 'KR', 'NA', 'KM']
        self.test_people_names = ['KA']
        self.transform = transform
        self.is_train = is_train  # train set or test set

        self.train_data = []
        self.train_classes = []
        self.test_data = []
        self.test_classes = []
        for person_name in self.train_people_names:
            img_file_names = os.listdir(os.path.join(self.img_dir_pre_path, person_name))
            for img_file_name in img_file_names:
                img = Image.open(os.path.join(self.img_dir_pre_path, person_name, img_file_name)) 
                if self.transform is not None:
                    img = self.transform(img)
                self.train_data.append(np.array(img)) # 256*256 的数据
                self.train_classes.append(self.classes_map[img_file_name[3:5]])
                    
        for person_name in self.test_people_names:
            img_file_names = os.listdir(os.path.join(self.img_dir_pre_path, person_name))
            for img_file_name in img_file_names:
                img = Image.open(os.path.join(self.img_dir_pre_path, person_name, img_file_name)) 
                if self.transform is not None:
                    img = self.transform(img)
                self.test_data.append(np.array(img)) # 256*256 的数据
                self.test_classes.append(self.classes_map[img_file_name[3:5]])
        print("train_num: ", len(self.train_data), " test_num:", len(self.test_data))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if index >= self.__len__():
            return None, None
        if self.is_train:
            img, cla = self.train_data[index], self.train_classes[index]
        else:
            img, cla = self.test_data[index], self.test_classes[index]
            
        return img, cla

    def __len__(self):
        if self.is_train:
            return len(self.train_data)
        else:
            return len(self.test_data)



In [14]:
train_img_size = 64
transform_train = transforms.Compose([
    transforms.Resize(input_img_size),
#     transforms.TenCrop(input_img_size),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_img_size = 64
transform_test = transforms.Compose([
    transforms.Resize(test_img_size),
    transforms.TenCrop(input_img_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

train_data = JAFFE(True, transform_train)
test_data = JAFFE(False, transform_test)

train_num:  190  test_num: 23
train_num:  190  test_num: 23


In [15]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

criterion = nn.MSELoss()
# 随机梯度下降
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)



In [16]:
# 重新定义一下进度条
Total_Bar_Length = 30
def progress_bar(cur, tot, msg):
    s = "\r["
    prog = int(float(Total_Bar_Length) * (cur + 1) / tot)
    rest = Total_Bar_Length - prog - 1
    s = s + "=" * prog + ">" + "." * rest + "]"
    s += " | " + msg
    if cur < tot - 1:
        print(s, end="")
    else:
        print(s)
        

In [17]:
# Training
def train(epoch):
    print('\n------------Epoch: %d-------------' % epoch)
    print('learning_rate: %s' % str(learning_rate))
    global Train_acc
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    cur_train_acc = 0.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        if use_cuda:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE, torch.float)
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
#         print(outputs)
#         print(targets)
        loss = criterion(outputs, targets)
        loss.backward()
#         utils.clip_gradient(optimizer, 0.1)
        optimizer.step()

        train_loss += loss.data
        _, predicted = torch.max(outputs.data, 1) # torch.max() 加上dim参数后，返回值为 max_value, max_value_index
        _, ground_value = torch.max(targets.data, 1)
        
        for i in range(len(predicted)):
            if predicted[i] == ground_value[i]:
                train_acc_map[predicted[i].item()] += 1
        
        total += targets.size(0)
        correct += predicted.eq(ground_value.data).cpu().sum()
        cur_train_acc = (100. * correct / total).item()

        progress_bar(batch_idx, len(train_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), cur_train_acc, correct, total))
        
    Train_acc = cur_train_acc
    if train_acc_map['best_acc'] < Train_acc:
        train_acc_map['best_acc'] = Train_acc
        train_acc_map['best_acc_epoch'] = epoch

def test(epoch):
    global Test_acc
    net.eval()
    PrivateTest_loss = 0
    correct = 0
    total = 0
    cur_test_acc = 0.
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            bs, ncrops, c, h, w = np.shape(inputs)
            inputs = inputs.view(-1, c, h, w)

            if use_cuda:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE, torch.float)
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops

            loss = criterion(outputs_avg, targets)
            PrivateTest_loss += loss.data
            _, predicted = torch.max(outputs_avg.data, 1)
            _, ground_value = torch.max(targets.data, 1)
            
            for i in range(len(predicted)):
                if predicted[i] == ground_value[i]:
                    test_acc_map[predicted[i].item()] += 1
                    
            total += targets.size(0)
            correct += predicted.eq(ground_value.data).cpu().sum()
            cur_test_acc = (100. * correct / total).item()

            progress_bar(batch_idx, len(test_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (PrivateTest_loss / (batch_idx + 1), cur_test_acc, correct, total))
        
    Test_acc = cur_test_acc
    if test_acc_map['best_acc'] < Test_acc:
        test_acc_map['best_acc'] = Test_acc
        test_acc_map['best_acc_epoch'] = epoch

In [18]:
train_acc_map = {'best_acc': 0, 'best_acc_epoch': -1, 0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
test_acc_map = {'best_acc': 0, 'best_acc_epoch': -1, 0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

start_epoch = 0
total_epoch = 1
for epoch in range(start_epoch, total_epoch):
    train(epoch)
#     for name,parameters in net.named_parameters():
#         print(name,':',parameters.size())
#         print(parameters)
#         break
    test(epoch)


------------Epoch: 0-------------
learning_rate: 0.01
[==============================>] | Loss: 0.123 | Acc: 14.000% (28/190)
[==============================>] | Loss: 0.123 | Acc: 17.000% (4/23)


In [19]:
print(train_acc_map)
print(test_acc_map)
for parameters in net.parameters():
    print(parameters.size())
    print(parameters[0][0][0])
    break

{'best_acc': 14, 'best_acc_epoch': 0, 0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 28}
{'best_acc': 17, 'best_acc_epoch': 0, 0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 4}
torch.Size([10, 1, 5, 5])
tensor([ 0.0259,  0.0413,  0.1918, -0.0401, -0.0909], device='cuda:0',
       grad_fn=<SelectBackward>)


以下部分是测试一下其他的已知net
---------

In [20]:
import torchvision

In [21]:
model = torchvision.models.alexnet(pretrained=False) #我们不下载预训练权重
net = model.to(DEVICE)
print(net)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [24]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
print(input)
print(target)
print(output)

tensor([[ 0.0581,  2.7572, -0.4979, -0.4385,  0.3852],
        [ 0.4089, -0.6164,  0.3215, -1.3282,  1.3405],
        [ 0.2028, -0.1411, -0.6483, -0.0541, -0.9250]], requires_grad=True)
tensor([1, 1, 4])
tensor(1.7164, grad_fn=<NllLossBackward>)


以下部分用于 de CK+和FER2013数据集的bug
----

In [5]:
import torch.nn as nn
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(3, 1)
 
    def forward(self, x):
        out = self.linear(x)
        return out

In [6]:
from utils.utils import num_of_parameters_of_net
net = LinearRegression()
print(num_of_parameters_of_net(net))

linear.weight : torch.Size([1, 3])
3
linear.bias : torch.Size([1])
1
4
